In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector,make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import ElasticNet
import sklearn

In [2]:
house_df = pd.read_csv(r'..\Datasets\Housing.csv')
house_df.head()

,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea
0,42000.0,5850,3,1,2,yes,no,yes,no,no,1,no
1,38500.0,4000,2,1,1,yes,no,no,no,no,0,no
2,49500.0,3060,3,1,1,yes,no,no,no,no,0,no
3,60500.0,6650,3,1,2,yes,yes,no,no,no,0,no
4,61000.0,6360,2,1,1,yes,no,no,no,no,0,no


In [3]:
x,y= house_df.iloc[:,1:], house_df['price']

### Preprocesing

In [4]:
ohe=OneHotEncoder(sparse=False, drop='first').set_output(transform='pandas')
ct1=make_column_transformer((ohe, make_column_selector(dtype_include='object')), remainder='passthrough',verbose_feature_names_out=False)
ct1=ct1.set_output(transform='pandas')

In [5]:
X_ohe=ct1.fit_transform(x)
best_model=ElasticNet(alpha=0.1, l1_ratio=0.75, random_state=25)
best_model.fit(X_ohe, y)

ElasticNet(alpha=0.1, l1_ratio=0.75, random_state=25)

### Inferencing

In [6]:
test_df=pd.read_csv(r'..\Datasets\tst_Housing.csv')
test_ohe=ct1.transform(test_df)
test_ohe.head()

,driveway_yes,recroom_yes,fullbase_yes,gashw_yes,airco_yes,prefarea_yes,lotsize,bedrooms,bathrms,stories,garagepl
0,1.0,0.0,1.0,0.0,0.0,0.0,5600,3,1,2,1
1,1.0,0.0,0.0,0.0,0.0,0.0,4355,2,1,1,0
2,1.0,0.0,0.0,0.0,0.0,0.0,3412,3,1,1,0
3,1.0,1.0,0.0,0.0,0.0,0.0,6666,2,2,2,0


In [7]:
predictions=best_model.predict(test_ohe)
predictions 

array([66465.70604475, 43208.12351626, 41814.57533902, 75988.14292848])

### Doing the above using pipelines

In [8]:
pipe1=Pipeline(steps=[('transform', ct1), ('model', best_model)])
pipe1.fit(x, y)
predictions=pipe1.predict(test_df)
predictions

array([66465.70604475, 43208.12351626, 41814.57533902, 75988.14292848])